# Naive Bayes Network

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:90% ! important; }<style>"))

#### Import dependencies

In [ ]:
# Import dependencies
# %pip install pgmpy 
# %pip install tabulate
# %pip install pandas
# %pip install networkx
# %pip install matplotlib

#### Load libraries

In [ ]:
# Load libraries
import pgmpy as pg
import tabulate as tb
import pandas as pd
import numpy as np
import missingno as mi
import networkx as nx
import matplotlib.pyplot as plt
import math

## Data import and cleaning

#### Import clinical data

In [ ]:
# Import data
df=pd.read_csv('symptoms.csv', sep=',', header=0)

# df.head()

In [ ]:
df.describe().transpose()

#### Define funtion to discretise data into catagories

In [ ]:
cough_labels = ['1. 0', '2. 1']
fever_labels = ['1. 0', '2. 1']
sob_labels = ['1. 0', '2. 1']
diarrhea_labels = ['1. 0', '2. 1']
fatigue_labels = ['1. 0', '2. 1']
headache_labels = ['1. 0', '2. 1']
loss_of_smell_labels = ['1. 0', '2. 1']
loss_of_taste_labels = ['1. 0', '2. 1']
runny_nose_labels = ['1. 0', '2. 1']
muscle_sore_labels = ['1. 0', '2. 1']
sore_throat_labels = ['1. 0', '2. 1']
covid_severity_labels = ['1. 1', '2. 2', '3. 3', '4. 4', '5. 5', '6. 6']

In [ ]:
def make_discrete(df):

    discrete_df = pd.DataFrame()

    discrete_df['cough'] = pd.cut(df['cough'], bins=2, labels=cough_labels, precision=2)
    discrete_df['fever'] = pd.cut(df['fever'], bins=2, labels=fever_labels, precision=2)
    discrete_df['sob'] = pd.cut(df['sob'], bins=2, labels=sob_labels, precision=2)
    discrete_df['diarrhea'] = pd.cut(df['diarrhea'], bins=2, labels=diarrhea_labels, precision=2)
    discrete_df['fatigue'] = pd.cut(df['fatigue'], bins=2, labels=fatigue_labels, precision=2)
    discrete_df['headache'] = pd.cut(df['headache'], bins=2, labels=headache_labels, precision=2)
    discrete_df['loss_of_smell'] = pd.cut(df['loss_of_smell'], bins=2, labels=loss_of_smell_labels, precision=2)
    discrete_df['loss_of_taste'] = pd.cut(df['loss_of_taste'], bins=2, labels=loss_of_taste_labels, precision=2)
    discrete_df['runny_nose'] = pd.cut(df['runny_nose'], bins=2, labels=runny_nose_labels, precision=2)
    discrete_df['muscle_sore'] = pd.cut(df['muscle_sore'], bins=2, labels=muscle_sore_labels, precision=2)
    discrete_df['sore_throat'] = pd.cut(df['sore_throat'], bins=2, labels=sore_throat_labels, precision=2)
    discrete_df['covid_severity'] = pd.cut(df['covid_severity'], bins=6, labels=covid_severity_labels, precision=2)

    discrete_df = discrete_df.astype('object')

    return discrete_df

#### Descritise data into catagories

In [ ]:
# Discretise clinical dataset
cat_df = make_discrete(df)

In [ ]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(cat_df, test_size=0.2, random_state=20)

In [ ]:
from itertools import combinations

import networkx as nx
from sklearn.metrics import f1_score
from pgmpy.models import BayesianModel, NaiveBayes, BayesianNetwork
from pgmpy.estimators import PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score, BicScore, BDeuScore
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator

In [ ]:
df.shape[1]

In [ ]:
model = NaiveBayes(feature_vars=['cough','fever','sob','diarrhea','fatigue','headache','loss_of_smell','loss_of_taste','runny_nose','muscle_sore','sore_throat'], dependent_var='covid_severity')

In [ ]:
model.fit(data=training_data, parent_node='covid_severity', estimator=BayesianEstimator)
bayes_model = BayesianNetwork(model)

bayes_model.fit(data=training_data,
    estimator=BayesianEstimator,
    prior_type='BDeu',
    complete_samples_only=True)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 8))
G = nx.DiGraph()
G.add_edges_from(model.edges)
G.add_nodes_from(model.nodes)

pos = nx.circular_layout(G)  # Use circular layout
DAG = G.to_directed()
nx.topological_sort(DAG)

nx.draw_networkx(
    G,
    pos=pos,
    with_labels=True,
    node_size=5000,
    arrowsize=30,
    alpha=0.7,
    font_weight="bold",
    node_color='#80DAE4',
    width=2.0,
    ax=ax
)

tt_g = G.subgraph(nodes=['covid_severity'])
nx.draw(
    tt_g,
    pos=pos,
    with_labels=False,
    arrowsize=0,
    node_size=7100,
    alpha=0.7,
    font_weight="bold",
    node_color='#EE3B3B',
    ax=ax
)

plt.savefig('naiveBayesNetwork.pdf', facecolor='w', bbox_inches='tight')
plt.show()


In [ ]:
bayes_model.get_cpds()

## Conditional Probability Table (CPT)

In [ ]:
print(f'Check model: {bayes_model.check_model()}\n')
for cpd in bayes_model.get_cpds():
    print(f'CPT of {cpd.variable}:')
    print(cpd * 100, '\n')